In [0]:
dbutils.fs.mount (
    source = "wasbs://retail@poc13storageretail13.blob.core.windows.net",
    mount_point = '/mnt/retailprojectadf',
    extra_configs = {"fs.azure.account.key.poc13storageretail13.blob.core.windows.net":"{key}"}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1628159609128113>:1
----> 1 dbutils.fs.mount (
      2     source = "wasbs://retail@poc13storageretail13.blob.core.windows.net",
      3     mount_point = '/mnt/retailprojectadf',
      4     extra_configs = {"fs.azure.account.key.poc13storageretail13.blob.core.windows.net":"dxWIwKF3g4zUGL+v7NShlWgHfdVObfS0JyP2aMAOq/lkwt6ClZrGIbmJPeBsHgpf1WjB7xSdAa6L+AStu6FfBQ=="}
      5 )

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 exc.__context__ = None
    363 exc.__cause__ = None
--> 364 raise exc

ExecutionError: An error occurred while calling o418.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/retailprojectadf; nested exception is: 
	java

In [0]:
dbutils.fs.ls('/mnt/retailprojectadf/Bronze/Transactions/')

Out[1]: [FileInfo(path='dbfs:/mnt/retailprojectadf/Bronze/Transactions/dbo.transactions.parquet', name='dbo.transactions.parquet', size=1757, modificationTime=1756762455000)]

In [0]:
df_transactions = spark.read.parquet('/mnt/retailprojectadf/Bronze/Transactions/')
# display(df_transactions)


transaction_id,customer_id,product_id,store_id,quantity,transaction_date
1,127,8,4,4,2025-03-31
2,105,3,4,5,2024-11-12
3,116,2,2,3,2025-05-01
4,120,8,1,1,2024-11-02
5,105,5,2,1,2025-03-17
6,110,7,3,5,2025-01-04
7,110,7,2,5,2025-01-01
8,126,7,5,2,2025-06-08
9,123,1,3,2,2024-10-08
10,124,2,2,5,2024-08-27


In [0]:
df_products = spark.read.parquet("/mnt/retailprojectadf/Bronze/Products/")
df_store = spark.read.parquet("/mnt/retailprojectadf/Bronze/Stores/")

# display(df_products)

In [0]:
df_customers = spark.read.parquet("/mnt/retailprojectadf/Bronze/Customers/manish040596/azure-data-engineer---multi-source/refs/heads/main/customers.parquet/")
# display(df_customers)

Cleaning the data in order to transfer it to silver layer from bronze


In [0]:
display(df_transactions)

transaction_id,customer_id,product_id,store_id,quantity,transaction_date
1,127,8,4,4,2025-03-31
2,105,3,4,5,2024-11-12
3,116,2,2,3,2025-05-01
4,120,8,1,1,2024-11-02
5,105,5,2,1,2025-03-17
6,110,7,3,5,2025-01-04
7,110,7,2,5,2025-01-01
8,126,7,5,2,2025-06-08
9,123,1,3,2,2024-10-08
10,124,2,2,5,2024-08-27


In [0]:
from pyspark.sql.functions import col 

df_transactions = df_transactions.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("int")
)

df_store = df_store.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

df_customers = df_customers.select(
    "customer_id" , "first_name", "last_name","email","city","registration_date"
).dropDuplicates(["customer_id"])

In [0]:
df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_store, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))

In [0]:

display(df_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999,Mega Plaza,Chennai,14997
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799,Tech World Outlet,Bangalore,1598
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799,Downtown Mini Store,Pune,2397
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499,Tech World Outlet,Bangalore,1996
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,Chennai,2023-06-28,Wireless Mouse,Electronics,799,Tech World Outlet,Bangalore,3995
2,5,105,5,1,2025-03-17,Riya,Singh,user105@example.com,Chennai,2023-06-28,Notebook Set,Stationery,149,High Street Store,Delhi,149
4,3,105,2,5,2024-11-12,Riya,Singh,user105@example.com,Chennai,2023-06-28,Yoga Mat,Fitness,499,Downtown Mini Store,Pune,2495
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999,Tech World Outlet,Bangalore,7996
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149,City Mall Store,Mumbai,596
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,Desk Organizer,Accessories,399,Mega Plaza,Chennai,1995


In [0]:
# DBTITLE 1,dump to adls location
silver_path = "/mnt/retailprojectadf/silver/"
df_silver.write.mode("overwrite").format("delta").save(silver_path)
# COMMAND ----------




In [0]:
# DBTITLE 1,create silver dataset
spark.sql(f"""
CREATE TABLE retail_silver_cleaned
USING DELTA
LOCATION '/mnt/retailprojectadf/silver/'
""")

Out[18]: DataFrame[]

In [0]:
%sql select * from retail_silver_cleaned

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999,Mega Plaza,Chennai,14997
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799,Tech World Outlet,Bangalore,1598
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799,Downtown Mini Store,Pune,2397
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499,Tech World Outlet,Bangalore,1996
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,Chennai,2023-06-28,Wireless Mouse,Electronics,799,Tech World Outlet,Bangalore,3995
2,5,105,5,1,2025-03-17,Riya,Singh,user105@example.com,Chennai,2023-06-28,Notebook Set,Stationery,149,High Street Store,Delhi,149
4,3,105,2,5,2024-11-12,Riya,Singh,user105@example.com,Chennai,2023-06-28,Yoga Mat,Fitness,499,Downtown Mini Store,Pune,2495
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999,Tech World Outlet,Bangalore,7996
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149,City Mall Store,Mumbai,596
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,Desk Organizer,Accessories,399,Mega Plaza,Chennai,1995


In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

gold_df = silver_df.groupBy(
    "transaction_date", "product_id", "product_name", "category", "store_id", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)


In [0]:
gold_path = "/mnt/retailprojectadf/Gold/"
gold_df.write.mode("overwrite").format("delta").save(gold_path)


In [0]:
display(gold_df)

transaction_date,product_id,product_name,category,store_id,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-15,7,Smartwatch,Electronics,5,Chennai,3,14997,1,14997.0
2025-03-16,8,Desk Organizer,Accessories,4,Pune,5,1995,1,1995.0
2025-05-26,5,Notebook Set,Stationery,1,Mumbai,4,596,1,596.0
2025-05-04,3,Yoga Mat,Fitness,3,Bangalore,4,1996,1,1996.0
2024-07-14,8,Desk Organizer,Accessories,4,Pune,4,1596,1,1596.0
2024-10-08,1,Wireless Mouse,Electronics,3,Bangalore,2,1598,1,1598.0
2025-06-08,7,Smartwatch,Electronics,5,Chennai,2,9998,1,9998.0
2024-10-10,8,Desk Organizer,Accessories,1,Mumbai,1,399,1,399.0
2024-07-17,1,Wireless Mouse,Electronics,4,Pune,5,3995,1,3995.0
2024-11-29,6,Water Bottle,Fitness,2,Delhi,4,1196,1,1196.0


In [0]:
spark.sql("""
          create table retail_gold_sales_summary
          using delta
          location '/mnt/retailprojectadf/Gold/'
          """)

Out[25]: DataFrame[]

In [0]:
%sql
select * from retail_gold_sales_summary

transaction_date,product_id,product_name,category,store_id,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-15,7,Smartwatch,Electronics,5,Chennai,3,14997,1,14997.0
2025-03-16,8,Desk Organizer,Accessories,4,Pune,5,1995,1,1995.0
2025-05-26,5,Notebook Set,Stationery,1,Mumbai,4,596,1,596.0
2025-05-04,3,Yoga Mat,Fitness,3,Bangalore,4,1996,1,1996.0
2024-07-14,8,Desk Organizer,Accessories,4,Pune,4,1596,1,1596.0
2024-10-08,1,Wireless Mouse,Electronics,3,Bangalore,2,1598,1,1598.0
2025-06-08,7,Smartwatch,Electronics,5,Chennai,2,9998,1,9998.0
2024-10-10,8,Desk Organizer,Accessories,1,Mumbai,1,399,1,399.0
2024-07-17,1,Wireless Mouse,Electronics,4,Pune,5,3995,1,3995.0
2024-11-29,6,Water Bottle,Fitness,2,Delhi,4,1196,1,1196.0
